<a href="https://colab.research.google.com/github/AI4Finance-LLC/FinRL-Library/blob/master/FinRL_ensemble_stock_trading_ICAIF_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Reinforcement Learning for Stock Trading from Scratch: Multiple Stock Trading Using Ensemble Strategy

Tutorials to use OpenAI DRL to trade multiple stocks using ensemble strategy in one Jupyter Notebook | Presented at ICAIF 2020

* This notebook is the reimplementation of our paper: Deep Reinforcement Learning for Automated Stock Trading: An Ensemble Strategy, using FinRL.
* Check out medium blog for detailed explanations: https://medium.com/@ai4finance/deep-reinforcement-learning-for-automated-stock-trading-f1dad0126a02
* Please report any issues to our Github: https://github.com/AI4Finance-LLC/FinRL-Library/issues
* **Pytorch Version** 



# Content

* [1. Problem Definition](#0)
* [2. Getting Started - Load Python packages](#1)
    * [2.1. Install Packages](#1.1)    
    * [2.2. Check Additional Packages](#1.2)
    * [2.3. Import Packages](#1.3)
    * [2.4. Create Folders](#1.4)
* [3. Download Data](#2)
* [4. Preprocess Data](#3)        
    * [4.1. Technical Indicators](#3.1)
    * [4.2. Perform Feature Engineering](#3.2)
* [5.Build Environment](#4)  
    * [5.1. Training & Trade Data Split](#4.1)
    * [5.2. User-defined Environment](#4.2)   
    * [5.3. Initialize Environment](#4.3)    
* [6.Implement DRL Algorithms](#5)  
* [7.Backtesting Performance](#6)  
    * [7.1. BackTestStats](#6.1)
    * [7.2. BackTestPlot](#6.2)   
    * [7.3. Baseline Stats](#6.3)   
    * [7.3. Compare to Stock Market Index](#6.4)             

<a id='0'></a>
# Part 1. Problem Definition

This problem is to design an automated trading solution for single stock trading. We model the stock trading process as a Markov Decision Process (MDP). We then formulate our trading goal as a maximization problem.

The algorithm is trained using Deep Reinforcement Learning (DRL) algorithms and the components of the reinforcement learning environment are:


* Action: The action space describes the allowed actions that the agent interacts with the
environment. Normally, a ∈ A includes three actions: a ∈ {−1, 0, 1}, where −1, 0, 1 represent
selling, holding, and buying one stock. Also, an action can be carried upon multiple shares. We use
an action space {−k, ..., −1, 0, 1, ..., k}, where k denotes the number of shares. For example, "Buy
10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or −10, respectively

* Reward function: r(s, a, s′) is the incentive mechanism for an agent to learn a better action. The change of the portfolio value when action a is taken at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio
values at state s′ and s, respectively

* State: The state space describes the observations that the agent receives from the environment. Just as a human trader needs to analyze various information before executing a trade, so
our trading agent observes many different features to better learn in an interactive environment.

* Environment: Dow 30 consituents


The data of the single stock that we will be using for this case study is obtained from Yahoo Finance API. The data contains Open-High-Low-Close price and volume.


<a id='1'></a>
# Part 2. Getting Started- Load Python Packages

<a id='1.1'></a>
## 2.1. Install all the packages through FinRL library


In [1]:
# ## install finrl library
# !pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git


<a id='1.2'></a>
## 2.2. Check if the additional packages needed are present, if not install them. 
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Import Packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.config import config
from finrl.marketdata.yahoodownloader import YahooDownloader
from finrl.preprocessing.preprocessors import FeatureEngineer
from finrl.preprocessing.data import data_split
from finrl.env.env_stocktrading import StockTradingEnv
from finrl.model.models import DRLAgent,DRLEnsembleAgent
from finrl.trade.backtest import get_baseline, backtest_stats, backtest_plot

from pprint import pprint

import sys
sys.path.append("../FinRL-Library")

import itertools

<a id='1.4'></a>
## 2.4. Create Folders

In [3]:
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

<a id='2'></a>
# Part 3. Download Data
Yahoo Finance is a website that provides stock data, financial news, financial reports, etc. All the data provided by Yahoo Finance is free.
* FinRL uses a class **YahooDownloader** to fetch data from Yahoo Finance API
* Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).




-----
class YahooDownloader:
    Provides methods for retrieving daily stock data from
    Yahoo Finance API

    Attributes
    ----------
        start_date : str
            start date of the data (modified from config.py)
        end_date : str
            end date of the data (modified from config.py)
        ticker_list : list
            a list of stock tickers (modified from config.py)

    Methods
    -------
    fetch_data()
        Fetches data from yahoo API


In [3]:
# from config.py start_date is a string
config.START_DATE

'2004-08-11'

In [4]:
print(config.DOW_30_TICKER)

['AAPL', 'MSFT', 'JPM', 'RTX', 'PG', 'GS', 'NKE', 'DIS', 'AXP', 'HD', 'INTC', 'WMT', 'IBM', 'MRK', 'UNH', 'KO', 'CAT', 'TRV', 'JNJ', 'CVX', 'MCD', 'VZ', 'CSCO', 'XOM', 'BA', 'MMM', 'PFE', 'WBA', 'DD']


In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
# from finrl.config.config import MISSING3
# df = YahooDownloader(start_date = '2006-01-01',
#                      end_date = '2021-06-11',
#                      ticker_list = config.DOW_30_TICKER).fetch_data()

In [7]:
# df.append(df2).append(df3).append(df4).append(df5)

In [17]:
import pickle
# df.drop('Unnamed: 0')with open('dji_df_2004-2021.pkl', 'rb') as f:
#     df = pickle.load(f)

df = pd.read_csv('/home/roman/Work/trading-bot/notebooks/dji_prices_2020_04_09.csv')
df

,Unnamed: 0,date,open,high,low,close,volume,tic,day
0,0,2006-01-03,2.585000,2.669643,2.580357,2.295634,807234400,AAPL,1
1,1,2006-01-03,51.700001,52.580002,51.049999,41.155041,7825700,AXP,1
2,2,2006-01-03,70.400002,70.599998,69.330002,50.119705,4943000,BA,1
3,3,2006-01-03,57.869999,58.110001,57.049999,38.086823,3697500,CAT,1
4,4,2006-01-03,17.209999,17.490000,17.180000,12.956775,55426000,CSCO,1
...,...,...,...,...,...,...,...,...,...
116020,116020,2021-06-10,233.100006,234.259995,232.130005,233.949997,4452500,V,3
116021,116021,2021-06-10,57.330002,57.610001,57.220001,57.340000,12013600,VZ,3
116022,116022,2021-06-10,53.799999,55.580002,53.570000,55.310001,6638000,WBA,3
116023,116023,2021-06-10,139.080002,140.190002,139.080002,139.880005,5459500,WMT,3


In [22]:
df = df.drop(['Unnamed: 0'], axis=1)

In [23]:
df = df.sort_values(['date', 'tic'])

In [24]:
df.head()

,date,open,high,low,close,volume,tic,day
0,2006-01-03,2.585000,2.669643,2.580357,2.295634,807234400,AAPL,1
1,2006-01-03,51.700001,52.580002,51.049999,41.155041,7825700,AXP,1
2,2006-01-03,70.400002,70.599998,69.330002,50.119705,4943000,BA,1
3,2006-01-03,57.869999,58.110001,57.049999,38.086823,3697500,CAT,1
4,2006-01-03,17.209999,17.490000,17.180000,12.956775,55426000,CSCO,1


In [27]:
df.tail()

,date,open,high,low,close,volume,tic,day
74625,2021-06-10,155.630005,155.889999,153.929993,154.020004,900500,TRV,3
74626,2021-06-10,57.330002,57.610001,57.220001,57.340000,12013600,VZ,3
74627,2021-06-10,53.799999,55.580002,53.570000,55.310001,6638000,WBA,3
74628,2021-06-10,139.080002,140.190002,139.080002,139.880005,5459500,WMT,3
74629,2021-06-10,63.610001,63.980000,62.250000,62.750000,27488700,XOM,3


In [28]:
df.shape

(74630, 8)

In [25]:
df.tic.unique()

array(['AAPL', 'AXP', 'BA', 'CAT', 'CSCO', 'CVX', 'DD', 'DIS', 'GS', 'HD',
       'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 'MRK', 'MSFT',
       'NKE', 'PFE', 'PG', 'RTX', 'TRV', 'UNH', 'VZ', 'WBA', 'WMT', 'XOM',
       'V'], dtype=object)

In [28]:
# import pickle
# with open('dji_df_2004-2021.pkl', 'wb') as f:
#     pickle.dump(df, f)

# Part 4: Preprocess Data
Data preprocessing is a crucial step for training a high quality machine learning model. We need to check for missing data and do feature engineering in order to convert the data into a model-ready state.
* Add technical indicators. In practical trading, various information needs to be taken into account, for example the historical stock prices, current holding shares, technical indicators, etc. In this article, we demonstrate two trend-following technical indicators: MACD and RSI.
* Add turbulence index. Risk-aversion reflects whether an investor will choose to preserve the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the financial turbulence index that measures extreme asset price fluctuation.

In [26]:
import pickle
with open('/home/roman/Work/trading-bot/notebooks/2004-2021_imputed_sentiment_final_df.pkl', 'rb') as f:
    daily_sentiment_df = pickle.load(f)

In [ ]:
with open('doc2vec_2004_2021_expanded_world_df.pkl', 'rb') as f:
    doc2vec_2004_2021_expanded_world_df = pickle.load(f)

In [27]:
daily_sentiment_df

,date,business_score,environment_score,politics_score,science_score,technology_score,world_score,business_magnitude,environment_magnitude,politics_magnitude,science_magnitude,technology_magnitude,world_magnitude
0,2004-08-11,-0.033333,-0.366667,-0.133333,0.066667,-0.033333,-0.333333,4.533333,9.400000,17.900000,3.500,4.900000,10.100000
1,2004-08-12,-0.166667,-0.100000,-0.366667,-0.100000,-0.066667,-0.233333,5.166667,15.433333,19.966667,5.300,10.066666,13.033334
2,2004-08-13,-0.300000,-0.100000,-0.066667,-0.100000,-0.133333,-0.100000,3.833333,13.000000,13.233334,5.300,2.266667,19.966667
3,2004-08-14,-0.233333,-0.233333,-0.300000,-0.100000,0.033333,-0.200000,9.400000,10.600000,16.100000,5.300,7.533334,13.933333
4,2004-08-15,-0.100000,-0.333333,-0.300000,-0.100000,-0.050000,-0.333333,8.600000,13.733334,19.200000,5.300,7.200000,5.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6141,2021-06-07,-0.152000,-0.111765,-0.313043,-0.050000,-0.208333,-0.226000,15.572000,10.935294,18.982609,6.175,14.133333,21.184000
6142,2021-06-08,-0.213793,-0.085714,-0.310526,-0.133333,-0.285714,-0.238000,14.465517,12.633334,25.405263,6.100,12.400000,19.104000
6143,2021-06-09,-0.200000,-0.084211,-0.256000,-0.150000,-0.200000,-0.234000,15.242857,11.794737,27.012000,7.000,8.875000,18.166000
6144,2021-06-10,-0.118519,-0.107692,-0.213636,-0.080000,-0.150000,-0.178000,14.151852,11.176923,23.786363,6.260,19.525000,17.912000


In [28]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = config.TECHNICAL_INDICATORS_LIST,
                    use_turbulence=True)

processed = fe.preprocess_data(df)

Successfully added technical indicators
Successfully added turbulence index


In [29]:
list(processed.columns)

['date',
 'open',
 'high',
 'low',
 'close',
 'volume',
 'tic',
 'day',
 'macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma',
 'turbulence']

In [30]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [31]:
processed_full.sample(5)

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
145404,2019-04-11,UNH,246.050003,246.919998,232.679993,226.985260,8223400.0,3.0,-1.950808,247.703096,229.131758,41.193396,-120.917290,34.839594,236.935162,245.672912,30.206898
56381,2011-02-25,INTC,21.520000,22.000000,21.459999,16.079168,53475700.0,4.0,0.095167,16.270748,15.492470,56.164736,77.443325,2.075169,15.754646,15.619415,26.364106
124973,2017-05-30,TRV,123.360001,124.230003,123.160004,111.790138,1139200.0,1.0,0.542851,111.698529,107.366933,59.091724,206.565871,35.819800,109.403681,109.583163,20.428736
26478,2008-06-03,MSFT,27.910000,28.309999,27.270000,20.466591,86616700.0,1.0,-0.296617,22.960626,20.394584,42.687388,-146.669316,20.872006,21.848735,21.691082,19.402154
13448,2007-03-27,GS,211.009995,211.729996,209.740005,172.573776,4806100.0,1.0,0.606949,176.135296,156.423773,53.246927,33.933005,17.471910,169.790019,171.120632,16.541630


In [32]:
processed_full['date'][0]

'2006-01-03'

<a id='4'></a>
# Part 5. Design Environment
Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a **Markov Decision Process (MDP)** problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.

The action space describes the allowed actions that the agent interacts with the environment. Normally, action a includes three actions: {-1, 0, 1}, where -1, 0, 1 represent selling, holding, and buying one share. Also, an action can be carried upon multiple shares. We use an action space {-k,…,-1, 0, 1, …, k}, where k denotes the number of shares to buy and -k denotes the number of shares to sell. For example, "Buy 10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or -10, respectively. The continuous action space needs to be normalized to [-1, 1], since the policy is defined on a Gaussian distribution, which needs to be normalized and symmetric.

In [33]:
config.TECHNICAL_INDICATORS_LIST

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma']

In [34]:
stock_dimension = len(processed_full.tic.unique())
state_space = 1 + 2*stock_dimension + len(config.TECHNICAL_INDICATORS_LIST)*stock_dimension + config.NUMBER_OF_DAILY_FEATURES
print(f"Stock Dimension: {stock_dimension}, User Features: {config.NUMBER_OF_USER_FEATURES}, State Space: {state_space}")


Stock Dimension: 30, User Features: 0, State Space: 313


In [35]:
env_kwargs = {
    "hmax": 100, 
    "initial_amount": 50_000_000/100, #Since in Indonesia the minimum number of shares per trx is 100, then we scaled the initial amount by dividing it with 100 
    "buy_cost_pct": 0.0019, #IPOT has 0.19% buy cost
    "sell_cost_pct": 0.0029, #IPOT has 0.29% sell cost
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": config.TECHNICAL_INDICATORS_LIST, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4,
    "print_verbosity":5
    
}

<a id='5'></a>
# Part 6: Implement DRL Algorithms
* The implementation of the DRL algorithms are based on **OpenAI Baselines** and **Stable Baselines**. Stable Baselines is a fork of OpenAI Baselines, with a major structural refactoring, and code cleanups.
* FinRL library includes fine-tuned standard DRL algorithms, such as DQN, DDPG,
Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

* In this notebook, we are training and validating 3 agents (A2C, PPO, DDPG) using Rolling-window Ensemble Method ([reference code](https://github.com/AI4Finance-LLC/Deep-Reinforcement-Learning-for-Automated-Stock-Trading-Ensemble-Strategy-ICAIF-2020/blob/80415db8fa7b2179df6bd7e81ce4fe8dbf913806/model/models.py#L92))

In [36]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
rebalance_window = 63 # rebalance_window is the number of days to retrain the model
validation_window = 63 # validation_window is the number of days to do validation and trading (e.g. if validation_window=63, then both validation and trading period will be 63 days)
train_start = '2006-01-03'
train_end = '2016-01-01'
val_test_start = '2016-01-01'
val_test_end = '2021-06-11'

ensemble_agent = DRLEnsembleAgent(df=processed_full,
                 train_period=(train_start,train_end),
                 val_test_period=(val_test_start,val_test_end),
                 rebalance_window=rebalance_window, 
                 validation_window=validation_window,
                 daily_features=daily_sentiment_df,
                 **env_kwargs)

In [41]:
A2C_model_kwargs = {
                    'n_steps': 5,
                    'ent_coef': 0.01,
                    'learning_rate': 0.0005
                    }

PPO_model_kwargs = {
                    "ent_coef":0.01,
                    "n_steps": 2048,
                    "learning_rate": 0.00025,
                    "batch_size": 128
                    }

DDPG_model_kwargs = {
                      "action_noise":"ornstein_uhlenbeck",
                      "buffer_size": 50_000,
                      "learning_rate": 0.000005,
                      "batch_size": 128
                    }

A2C2_model_kwargs = {
                    'n_steps': 5,
                    'ent_coef': 0.01,
                    'learning_rate': 0.001
                    }

PPO2_model_kwargs = {
                    "ent_coef":0.01,
                    "n_steps": 2048,
                    "learning_rate": 0.0005,
                    "batch_size": 128
                    }

DDPG2_model_kwargs = {
                      "action_noise":"ornstein_uhlenbeck",
                      "buffer_size": 50_000,
                      "learning_rate": 0.00001,
                      "batch_size": 128
                    }

timesteps_dict = {'a2c' : 50_000, 
                 'ppo' : 50_000, 
                 'ddpg' : 25_000,
                  'a2c2' : 25_000, 
                 'ppo2' : 25_000, 
                 'ddpg2' : 12_000
                 }

In [ ]:
import time
start = time.time()# print([self.state[0]])
        # print(self.data.close.values.tolist())
        # print(list(self.state[(self.stock_dim+1):(self.stock_dim*2+1)]))
        # print(sum([self.data[tech].values.tolist() for tech in self.tech_indicator_list ], []) )
        # user_features_columns = self.data.columns[-config.NUMBER_OF_USER_FEATURES:]
        # print(self.data[user_features_columns].values[0])
df_summary = ensemble_agent.run_ensemble_strategy(A2C_model_kwargs, PPO_model_kwargs, DDPG_model_kwargs, A2C2_model_kwargs, PPO2_model_kwargs, DDPG2_model_kwargs, timesteps_dict)
time_elapsed = time.time()-start
print(time_elapsed)

============Start Ensemble Strategy============
39.25064963834649
turbulence_threshold:  458.4056541260132
======Model training from:  2006-01-03 to  2016-01-04
======A2C Training========
{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0005}
Using cuda device
Logging to tensorboard_log/a2c/a2c_126_16
------------------------------------
| time/                 |          |
|    fps                | 156      |
|    iterations         | 100      |
|    time_elapsed       | 3        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -42.8    |
|    explained_variance | 0.104    |
|    learning_rate      | 0.0005   |
|    n_updates          | 99       |
|    policy_loss        | -30.6    |
|    std                | 1.01     |
|    value_loss         | 1.01     |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 157      |
|    iterations         | 20

------------------------------------
| time/                 |          |
|    fps                | 157      |
|    iterations         | 1500     |
|    time_elapsed       | 47       |
|    total_timesteps    | 7500     |
| train/                |          |
|    entropy_loss       | -43.2    |
|    explained_variance | 0        |
|    learning_rate      | 0.0005   |
|    n_updates          | 1499     |
|    policy_loss        | -8.06    |
|    std                | 1.02     |
|    value_loss         | 0.321    |
------------------------------------
------------------------------------
| environment/          |          |
|    portfolio_value    | 7.94e+05 |
|    total_cost         | 1.64e+05 |
|    total_reward       | 2.94e+05 |
|    total_reward_pct   | 58.8     |
|    total_trades       | 53109    |
| time/                 |          |
|    fps                | 157      |
|    iterations         | 1600     |
|    time_elapsed       | 50       |
|    total_timesteps    | 8000     |
|

------------------------------------
| time/                 |          |
|    fps                | 158      |
|    iterations         | 2900     |
|    time_elapsed       | 91       |
|    total_timesteps    | 14500    |
| train/                |          |
|    entropy_loss       | -43.5    |
|    explained_variance | 5.96e-08 |
|    learning_rate      | 0.0005   |
|    n_updates          | 2899     |
|    policy_loss        | -21.9    |
|    std                | 1.03     |
|    value_loss         | 0.389    |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 158      |
|    iterations         | 3000     |
|    time_elapsed       | 94       |
|    total_timesteps    | 15000    |
| train/                |          |
|    entropy_loss       | -43.6    |
|    explained_variance | 0        |
|    learning_rate      | 0.0005   |
|    n_updates          | 2999     |
|    policy_loss        | 66.1     |
|

------------------------------------
| time/                 |          |
|    fps                | 152      |
|    iterations         | 4300     |
|    time_elapsed       | 140      |
|    total_timesteps    | 21500    |
| train/                |          |
|    entropy_loss       | -44      |
|    explained_variance | 0.567    |
|    learning_rate      | 0.0005   |
|    n_updates          | 4299     |
|    policy_loss        | 16.4     |
|    std                | 1.05     |
|    value_loss         | 0.36     |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 152      |
|    iterations         | 4400     |
|    time_elapsed       | 144      |
|    total_timesteps    | 22000    |
| train/                |          |
|    entropy_loss       | -44.1    |
|    explained_variance | 0.0339   |
|    learning_rate      | 0.0005   |
|    n_updates          | 4399     |
|    policy_loss        | -31.6    |
|

------------------------------------
| time/                 |          |
|    fps                | 150      |
|    iterations         | 5700     |
|    time_elapsed       | 189      |
|    total_timesteps    | 28500    |
| train/                |          |
|    entropy_loss       | -44.7    |
|    explained_variance | 0        |
|    learning_rate      | 0.0005   |
|    n_updates          | 5699     |
|    policy_loss        | -26      |
|    std                | 1.07     |
|    value_loss         | 0.841    |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 150      |
|    iterations         | 5800     |
|    time_elapsed       | 192      |
|    total_timesteps    | 29000    |
| train/                |          |
|    entropy_loss       | -44.8    |
|    explained_variance | 0        |
|    learning_rate      | 0.0005   |
|    n_updates          | 5799     |
|    policy_loss        | -38.2    |
|

...

------------------------------------
| time/                 |          |
|    fps                | 134      |
|    iterations         | 17500    |
|    time_elapsed       | 648      |
|    total_timesteps    | 87500    |
| train/                |          |
|    entropy_loss       | -49.3    |
|    explained_variance | 0.627    |
|    learning_rate      | 0.0005   |
|    n_updates          | 17499    |
|    policy_loss        | -24.9    |
|    std                | 1.26     |
|    value_loss         | 0.386    |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 134      |
|    iterations         | 17600    |
|    time_elapsed       | 651      |
|    total_timesteps    | 88000    |
| train/                |          |
|    entropy_loss       | -49.4    |
|    explained_variance | 0.0698   |
|    learning_rate      | 0.0005   |
|    n_updates          | 17599    |
|    policy_loss        | 28.4     |
|

------------------------------------
| time/                 |          |
|    fps                | 135      |
|    iterations         | 18900    |
|    time_elapsed       | 699      |
|    total_timesteps    | 94500    |
| train/                |          |
|    entropy_loss       | -49.6    |
|    explained_variance | 0        |
|    learning_rate      | 0.0005   |
|    n_updates          | 18899    |
|    policy_loss        | 30.7     |
|    std                | 1.27     |
|    value_loss         | 0.48     |
------------------------------------
------------------------------------
| environment/          |          |
|    portfolio_value    | 1.46e+06 |
|    total_cost         | 1.89e+04 |
|    total_reward       | 9.61e+05 |
|    total_reward_pct   | 192      |
|    total_trades       | 53510    |
| time/                 |          |
|    fps                | 135      |
|    iterations         | 19000    |
|    time_elapsed       | 703      |
|    total_timesteps    | 95000    |
|

-----------------------------------------
| environment/            |             |
|    portfolio_value      | 3.67e+05    |
|    total_cost           | 4.97e+05    |
|    total_reward         | -1.33e+05   |
|    total_reward_pct     | -26.6       |
|    total_trades         | 80083       |
| time/                   |             |
|    fps                  | 140         |
|    iterations           | 3           |
|    time_elapsed         | 43          |
|    total_timesteps      | 6144        |
| train/                  |             |
|    approx_kl            | 0.017783215 |
|    clip_fraction        | 0.215       |
|    clip_range           | 0.2         |
|    entropy_loss         | -42.7       |
|    explained_variance   | -0.0219     |
|    learning_rate        | 0.00025     |
|    loss                 | 0.16        |
|    n_updates            | 20          |
|    policy_gradient_loss | -0.0433     |
|    std                  | 1.01        |
|    value_loss           | 1.28  

-----------------------------------------
| environment/            |             |
|    portfolio_value      | 4.65e+05    |
|    total_cost           | 4.72e+05    |
|    total_reward         | -3.55e+04   |
|    total_reward_pct     | -7.1        |
|    total_trades         | 78471       |
| time/                   |             |
|    fps                  | 141         |
|    iterations           | 12          |
|    time_elapsed         | 174         |
|    total_timesteps      | 24576       |
| train/                  |             |
|    approx_kl            | 0.030765755 |
|    clip_fraction        | 0.304       |
|    clip_range           | 0.2         |
|    entropy_loss         | -43.3       |
|    explained_variance   | 0.109       |
|    learning_rate        | 0.00025     |
|    loss                 | -0.11       |
|    n_updates            | 110         |
|    policy_gradient_loss | -0.0383     |
|    std                  | 1.03        |
|    value_loss           | 1.02  

----------------------------------------
| environment/            |            |
|    portfolio_value      | 8.29e+05   |
|    total_cost           | 4.61e+05   |
|    total_reward         | 3.29e+05   |
|    total_reward_pct     | 65.8       |
|    total_trades         | 77597      |
| time/                   |            |
|    fps                  | 142        |
|    iterations           | 21         |
|    time_elapsed         | 302        |
|    total_timesteps      | 43008      |
| train/                  |            |
|    approx_kl            | 0.03207866 |
|    clip_fraction        | 0.323      |
|    clip_range           | 0.2        |
|    entropy_loss         | -43.9      |
|    explained_variance   | -0.0819    |
|    learning_rate        | 0.00025    |
|    loss                 | 0.106      |
|    n_updates            | 200        |
|    policy_gradient_loss | -0.0384    |
|    std                  | 1.05       |
|    value_loss           | 1.6        |
----------------

In [82]:
df_summary

,Iter,Val Start,Val End,Model Used,A2C Sharpe,PPO Sharpe,DDPG Sharpe
0,126,2016-01-04,2016-04-05,PPO,0.127394,0.303177,0.264977
1,189,2016-04-05,2016-07-05,A2C,0.12121,-0.002137,0.070818
2,252,2016-07-05,2016-10-03,DDPG,-0.042202,-0.045131,0.151223
3,315,2016-10-03,2017-01-03,DDPG,0.573297,0.566399,0.705078
4,378,2017-01-03,2017-04-04,DDPG,-0.122285,-0.138578,-0.031246
5,441,2017-04-04,2017-07-05,DDPG,0.150122,0.098266,0.268507
6,504,2017-07-05,2017-10-03,A2C,0.30975,0.240212,0.113961
7,567,2017-10-03,2018-01-03,A2C,0.832907,0.487907,0.633911
8,630,2018-01-03,2018-04-05,A2C,-0.038953,-0.120097,-0.115019
9,693,2018-04-05,2018-07-05,DDPG,-0.208591,-0.177387,0.055126


In [83]:
del fe

NameError: name 'fe' is not defined

In [84]:
import dill
dill.dump_session('17_year_guardian_sentiment_fixed_fixed_dow.db')

PicklingError: Can't pickle <class 'finrl.preprocessing.preprocessors.FeatureEngineer'>: it's not the same object as finrl.preprocessing.preprocessors.FeatureEngineer

In [ ]:
processed_full

In [148]:
import pickle
with open('sentiment_1.pkl', 'wb') as f:
    pickle.dump(processed_full, f)

In [85]:
import pickle
with open('17_year_guardian_sentiment_fixed_fixed_dow_df_summary.pkl', 'wb') as f:
    pickle.dump(df_summary, f)

In [86]:
with open('17_year_guardian_sentiment_fixed_fixed_dow_processed_full.pkl', 'wb') as f:
    pickle.dump(processed_full, f)

In [92]:
with open('17_year_guardian_sentiment_fixed_fixed_dow_ensemble_agent.pkl', 'wb') as f:
    pickle.dump(ensemble_agent, f)

PicklingError: Can't pickle <class 'finrl.env.env_stocktrading.StockTradingEnv'>: it's not the same object as finrl.env.env_stocktrading.StockTradingEnv